In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-17 16:14:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-17 16:14:21 (6.50 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [5]:
from pyspark.sql.functions import to_date

In [9]:
vine_df = df.filter("total_votes>=20").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [10]:
vine_Review_df = vine_df.filter("helpful_votes/total_votes>=0.5").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_Review_df.show(15)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [11]:
vine_only_y_df = vine_Review_df.filter("vine == 'Y'").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_only_y_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3JIGR122X50ZV|          5|         2280|       2314|   Y|                N|
|R3KV8P8WLD2KKC|          5|          263|        286|   Y|                N|
|R2LFY2ZPJ3J7RH|          4|           36|         38|   Y|                N|
|R3P01OXMA0U0CA|          5|          114|        117|   Y|                N|
|R2SCMXBD9OQP8Z|          2|           22|         26|   Y|                N|
|R2FGS9ECOK5IQN|          5|           22|         23|   Y|                N|
| R1V9CO41XS6QY|          2|           56|         58|   Y|                N|
|R1LEP1JGXRY2YT|          2|           18|         20|   Y|                N|
|R1ZWQJ3ZBSIZ1K|          4|           19|         21|   Y|                Y|
| RX1G4FUE5HZGN|          3|           52|         58|   Y|     

In [12]:
n_Vine_only_df = vine_Review_df.filter("vine == 'N'").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
n_Vine_only_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [13]:
total_paid_review = vine_only_y_df.count()
total_paid_review

969

In [15]:
total_unpaid_review = n_Vine_only_df.count()
total_unpaid_review

43745

In [16]:
total_five_star_review = vine_only_y_df.filter("star_rating=5").count()
total_five_star_review

430

In [18]:
total_five_star_review_unpaid = n_Vine_only_df.filter("star_rating=5").count()
total_five_star_review_unpaid

19233

In [19]:
paid_percentage = (total_five_star_review/total_paid_review)*100
paid_percentage

44.375644994840044

In [21]:
unpaid_percentage = (total_five_star_review_unpaid/total_unpaid_review)*100
unpaid_percentage

43.96616756200709